In [ ]:
from pathlib import Path
from datautils.widgets import IconExtractor


path = Path("D:/repos/psicon/notebooks/temp/tileset")
ie = IconExtractor(path)
display(ie)

#image = ie._annotated_image.value



# from datautils.image import to_pil

# #display(to_pil(image))

# from datautils.image import iconsheet_extract
# import torch


# result = iconsheet_extract(image.cuda(), tolerance=0.1, dilation=8, background_color=torch.tensor([0,0,0,0]), return_foreground_mask=True)
# print(result.foreground_mask.shape)
# display(to_pil(result.foreground_mask.float()))


In [ ]:
from datautils.widgets import Image
import torch

image = Image(torch.zeros((3,128,128)), click_callback=print)
display(image)
image[:,10:20] = 1.0
image.refresh() # must be called after setting values with __setitem__!
image.value = torch.ones((3,50,50)) / 2

In [ ]:
from datautils.widgets import AnnotatedImage
import torch
image = AnnotatedImage(torch.zeros((3,128,128)), click_callback=print)
display(image)
image[:,10:20] = 1.0
image.refresh() # must be called after setting values with __setitem__!
image.value = torch.ones((3,50,50)) / 2
image.boxes = torch.tensor([[10,10,20,20],[12,15,40,30]])


In [ ]:
from datautils.widgets import IconGrid, AnnotatedImage
import torch
import time
buffer = torch.ones((4,128,128)) / 2
buffer[:,10:20] = 1.0
boxes = torch.tensor([[10,10,20,20],[12,15,40,30]])

image = IconGrid(buffer, boxes=boxes, click_callback=print, background_color=(255,0,0))
aimage = AnnotatedImage(buffer, boxes=boxes, click_callback=print)

display(image)
display(aimage)



In [ ]:
from datautils.widgets import Swatch

# Test cell for Swatch widget
from IPython.display import display

sw = Swatch("red")
display(sw)
sw.color = (0.0, 1.0, 0.0)  # green
display(sw)
